# 1.层级索引

我们接触的都是一维数据和二维数据，用 Pandas 的 Series 和 DataFrame 对
象就可以存储。但我们也经常会遇到存储多维数据的需求，数据索引超过一两个键。因
此，Pandas 提供了 Panel 和 Panel4D 对象解决三维数据与四维数据（详情请参见 3.7 节）。
而在实践中，更直观的形式是通过层级索引（hierarchical indexing，也被称为多级索引，
multi-indexing）配合多个有不同等级（level）的一级索引一起使用，这样就可以将高维数
组转换成类似一维 Series 和二维 DataFrame 对象的形式。

### 1.1 多级索引 Series

In [3]:
import pandas as pd
import numpy as np

index = [('California', 2000), ('California', 2010),
('New York', 2000), ('New York', 2010),
('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
18976457, 19378102,
20851820, 25145561]
pop = pd.Series(populations, index=index)
print("1",pop)
# 通过元组构成的多级索引，你可以直接在 Series 上取值或用切片查询数据：
print("2",pop[('California', 2010):('Texas', 2000)])

print("3",pop[[i for i in pop.index if i[1] == 2010]])

1 (California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64
2 (California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64
3 (California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64


### 1.2 Pandas 多级索引
用元组表示索引其实是多级索引的基础，Pandas
的 MultiIndex 类型提供了更丰富的操作方法。我们可以用元组创建一个多级索引，如
下所示：

In [4]:
index = pd.MultiIndex.from_tuples(index)
print(index)

pop = pop.reindex(index)
print(pop)

print(pop[:, 2010])

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64
California    37253956
New York      19378102
Texas         25145561
dtype: int64


### 1.3. 高维数据的多级索引
我们其实完全可以用一个带行列索引的简单 DataFrame 代替前面的多
级索引。其实 Pandas 已经实现了类似的功能。 unstack() 方法可以快速将一个多级索引的
Series 转化为普通索引的 DataFrame ：

In [5]:
pop_df = pop.unstack()
print(pop_df)
# stack() 方法实现相反的效果
print(pop_df.stack())

pop_df = pd.DataFrame({'total': pop,
'under18': [9267089, 9284094,
4687374, 4318033,
5906301, 6879014]})
print(pop_df)

f_u18 = pop_df['under18'] / pop_df['total']
print(f_u18.unstack())

                2000      2010
California  33871648  37253956
New York    18976457  19378102
Texas       20851820  25145561
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64
                    total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014
                2000      2010
California  0.273594  0.249211
New York    0.247010  0.222831
Texas       0.283251  0.273568


# 2 多级索引的创建方法
为 Series 或 DataFrame 创建多级索引最直接的办法就是将 index 参数设置为至少二维的索
引数组

In [6]:
df = pd.DataFrame(np.random.rand(4, 2),
index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
columns=['data1', 'data2'])
print(df)

# 如果你把将元组作为键的字典传递给 Pandas， Pandas 也会默认转换为 MultiIndex
data = {('California', 2000): 33871648,
('California', 2010): 37253956,
('Texas', 2000): 20851820,
('Texas', 2010): 25145561,
('New York', 2000): 18976457,
('New York', 2010): 19378102}
print(pd.Series(data))


        data1     data2
a 1  0.916558  0.625090
  2  0.456914  0.727619
b 1  0.936982  0.249786
  2  0.627208  0.325081
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64


### 2.1 显式地创建多级索引


In [7]:
# 你可以通过一个有不同等级的若干简单数组组成的列表来构建 MultiIndex ：
print(pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]]))
# 也可以通过包含多个索引值的元组构成的列表创建 MultiIndex ：
print(pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)]))
# 还可以用两个索引的笛卡尔积（Cartesian product）创建 MultiIndex ：
print(pd.MultiIndex.from_product([['a', 'b'], [1, 2]]))

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )
MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )
MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )


### 2.2. 多级索引的等级名称
给 MultiIndex 的等级加上名称会为一些操作提供便利。你可以在前面任何一个 MultiIndex
构造器中通过 names 参数设置等级名称，也可以在创建之后通过索引的 names 属性来修改名称：

In [8]:
pop.index.names = ['state', 'year']
print(pop)

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64


###### 2.1.2. 多级索引的等级名称